In [1]:
from nbed.driver import NbedDriver
from nbed.ham_builder import HamiltonianBuilder
from openfermion import get_sparse_operator
import scipy as sp
from openfermion.utils import count_qubits
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
from nbed.mol_plot import Draw_molecule
import pandas as pd
import pickle
from nbed.ham_converter import HamiltonianConverter

In [2]:
current_dir = os.getcwd()
structure_dir =  os.path.join(current_dir, "molecular_structures")

In [3]:
# order by R_length

list_files = [ filename for filename in os.listdir(structure_dir) if filename.startswith('act_stretch_H2O')]

all_xyz_files = sorted(list_files, 
                       key=lambda text: int(text.partition("H2O_")[2].partition(".xyz")[0]))
all_xyz_files

['act_stretch_H2O_40.xyz',
 'act_stretch_H2O_60.xyz',
 'act_stretch_H2O_79.xyz',
 'act_stretch_H2O_100.xyz',
 'act_stretch_H2O_120.xyz',
 'act_stretch_H2O_150.xyz',
 'act_stretch_H2O_200.xyz',
 'act_stretch_H2O_300.xyz',
 'act_stretch_H2O_400.xyz',
 'act_stretch_H2O_500.xyz']

In [4]:
list_file_paths = []
for xyz_filename in all_xyz_files:
    list_file_paths.append(os.path.join(structure_dir, xyz_filename))
list_file_paths

['/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_40.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_60.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_79.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_100.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_120.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_150.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_200.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_300.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_structures/act_stretch_H2O_400.xyz',
 '/home/lex/Documents/PhD/Nbed/projects/water_correlated/molecular_

In [5]:
with open(list_file_paths[0], 'r') as infile1:
    test_xyz = infile1.read()
    
print(test_xyz)

3
 
H	0.3751747	0.0	0.1387225
O	0.0	0.0	0.0
H	-0.7493682	0.0	0.2770822



In [6]:
basis = 'STO-3G'
xc_functional = 'B3LYP'
convergence = 1e-6
charge = 0
mu_level_shift=1e6
run_ccsd_emb = True
run_fci_emb = True
max_ram_memory = 8_000
pyscf_print_level = 1
unit = 'angstrom'


occupied_threshold = 0.95
virtual_threshold = 0.95
max_hf_cycles=100_000
max_dft_cycles = 100_000
transform = 'jordan_wigner'

In [7]:
n_active_atoms = 2
localization = 'ibo' #ibo, spade

projector = 'both' #'both', 'huzinaga'
init_huzinaga_rhf_with_mu = True

In [8]:
def get_hamiltonian_from_driver(rhf_pyscf, energy_shift, qubit_transform):
    # n_qubits = 2*rhf_pyscf.mol.nao
    qham = HamiltonianBuilder(
                            scf_method=rhf_pyscf,
                            constant_e_shift=energy_shift,
                            transform=qubit_transform,
                            ).build(n_qubits=None, taper=False)
    return qham

In [9]:
global_dft_list=[]
global_fci_list=[]
global_H_diag_list=[]
global_qubit_H_length_list=[]
global_qubit_H_nqubits=[]
global_H = []

huz_rhf_list=[]
huz_ccsd_list=[]
huz_fci_list=[]
huz_H_diag_list=[]
huz_qubit_H_length_list=[]
huz_H_nqubits=[]
huz_embed_H = []

mu_rhf_list=[]
mu_ccsd_list=[]
mu_fci_list=[]
mu_H_diag_list=[]
mu_qubit_H_length_list=[]
mu_H_nqubits=[]
mu_embed_H = []

N_active_MOs = []

for geometry_path in tqdm(list_file_paths):
#     try:
    driver = NbedDriver(
            geometry=geometry_path,
            n_active_atoms=n_active_atoms,
            basis=basis,
            xc_functional=xc_functional,
            projector=projector,
            localization=localization,
            convergence=convergence,
            charge=charge,
            mu_level_shift=mu_level_shift,
            run_ccsd_emb=run_ccsd_emb,
            run_fci_emb=run_fci_emb,
            max_ram_memory=max_ram_memory,
            pyscf_print_level=pyscf_print_level,
            unit=unit,
            occupied_threshold=occupied_threshold,
            virtual_threshold=virtual_threshold,
            max_hf_cycles=max_hf_cycles,
            max_dft_cycles=max_dft_cycles,
            _init_huzinaga_rhf_with_mu = init_huzinaga_rhf_with_mu)
#     except:
#         print('error for:')
#         print(geometry_path)
#         print()
#         continue

    ## global
    global_dft_list.append(driver._global_rks.e_tot)
    global_fci_list.append(driver._global_fci.e_tot)
    
    global_qubit_Ham = get_hamiltonian_from_driver(driver._global_hf,
                                           driver._global_hf.energy_nuc(), 
                                           transform)
    
    global_H_sparse = get_sparse_operator(global_qubit_Ham)
    E_fci =  sp.sparse.linalg.eigsh(global_H_sparse, k=1, which='SA')[0][0] 
    global_H_diag_list.append(E_fci)

    global_qubit_H_length_list.append(len(global_qubit_Ham.terms))
    global_qubit_H_nqubits.append(count_qubits(global_qubit_Ham))
    global_H.append(global_qubit_Ham)

    ##
    N_active_MOs.append(len(driver.localized_system.active_MO_inds))
    
    # HUZ
    huz_rhf_list.append(driver._huzinaga['e_rhf'])
    huz_ccsd_list.append(driver._huzinaga['e_ccsd'])
    huz_fci_list.append(driver._huzinaga['e_fci'])

    huz_classical = driver._huzinaga['classical_energy']
    huz_qubit_Ham = get_hamiltonian_from_driver(driver._huzinaga["scf"],
                                        huz_classical, #+ driver._huzinaga["scf"].energy_nuc(), 
                                        transform)
    
    huz_H_sparse = get_sparse_operator(huz_qubit_Ham)
    huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]
    huz_H_diag_list.append(huz_e_emb)

    huz_qubit_H_length_list.append(len(huz_qubit_Ham.terms))
    huz_nqubits = count_qubits(huz_qubit_Ham)
    huz_H_nqubits.append(huz_nqubits)
    huz_embed_H.append(huz_qubit_Ham)
    

    # mu shift
    mu_rhf_list.append(driver._mu['e_rhf'])
    mu_ccsd_list.append(driver._mu['e_ccsd'])
    mu_fci_list.append(driver._mu['e_fci'])
    
    mu_classical = driver._mu['classical_energy']
    mu_qubit_Ham = get_hamiltonian_from_driver(driver._mu["scf"],
                                        mu_classical, #+ driver._mu["scf"].energy_nuc(), 
                                        transform)
    
    mu_H_sparse = get_sparse_operator(mu_qubit_Ham)
    mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]
    mu_H_diag_list.append(mu_e_emb)
    
    mu_nqubits = count_qubits(mu_qubit_Ham)
    mu_qubit_H_length_list.append(len(mu_qubit_Ham.terms))
    mu_H_nqubits.append(mu_nqubits)
    mu_embed_H.append(mu_qubit_Ham)
    
    del driver



  0%|          | 0/10 [00:00<?, ?it/s]

 Iterative localization: IB/P4/2x2, 5 iter; Final gradient 1.56e-13


KeyboardInterrupt: 

In [ ]:
np.allclose(global_fci_list, global_H_diag_list)

In [ ]:
exp_data = {
    'global DFT': global_dft_list, 
    
    'global FCI': global_fci_list,
    'huz emb FCI': huz_H_diag_list,
    'mu emb FCI': mu_H_diag_list,
    
    'global H': global_qubit_H_length_list,
    'huz H emb': huz_qubit_H_length_list,
    'mu H emb': mu_qubit_H_length_list,
    
    'global n qubits': global_qubit_H_nqubits,
    'huz n qubits': huz_H_nqubits,
    'mu n qubits': mu_H_nqubits,
    'N active MOs': N_active_MOs,
    
}

df = pd.DataFrame(data=exp_data)

df

In [ ]:
csv_filename = 'active_OH_stretch_results_IBO.csv' 
df.to_csv(os.path.join(current_dir, csv_filename), index=False)

In [ ]:
exp_data['bond_length'] =  [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]
exp_data['global_H'] = [HamiltonianConverter(H)._intermediate for H in global_H]
exp_data['huz_embed_H'] = [HamiltonianConverter(H)._intermediate for H in huz_embed_H]
exp_data['mu_embed_H'] = [HamiltonianConverter(H)._intermediate for H in mu_embed_H]


In [ ]:
# import results
csv_filename = 'active_OH_stretch_results_IBO.csv' 
df = pd.read_csv(os.path.join(current_dir, csv_filename))

global_dft_list = np.array(df['global DFT'])

global_fci_list = np.array(df['global FCI'])
huz_H_diag_list = np.array(df['huz emb FCI'])
mu_H_diag_list = np.array(df['mu emb FCI'])

global_qubit_H_length_list = np.array(df['global H'])
huz_qubit_H_length_list = np.array(df['huz H emb'])
mu_qubit_H_length_list = np.array(df['mu H emb'])

global_qubit_H_nqubits = np.array(df['global n qubits'])
huz_H_nqubits = np.array(df['huz n qubits'])
mu_H_nqubits = np.array(df['mu n qubits'])
N_active_MOs = np.array(df['N active MOs'])



In [ ]:
%matplotlib inline

R_list =  [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]


plt.plot(R_list, global_dft_list, label='global DFT', color='b')
plt.plot(R_list, global_fci_list, label='global FCI', color='orange')
plt.plot(R_list, global_fci_list, label='diag global H', linestyle='dashed', color='darkred')
plt.plot(R_list, huz_H_diag_list, label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(R_list, mu_H_diag_list, label='mu emb', linestyle='-', marker='x', color='r')

plt.xlabel('OH bond length (angstrom)')
plt.ylabel('energy (Ha)')
plt.grid(True)
plt.legend()
plt.savefig("test_energy.png", dpi=300)
plt.show()

In [ ]:
%matplotlib inline

R_list =  [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]


plt.plot(R_list, abs(np.array(global_dft_list) - np.array(global_fci_list)), label='global DFT', color='b', marker='+')
plt.plot(R_list, abs(np.array(huz_H_diag_list) - np.array(global_fci_list)), label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(R_list, abs(np.array(mu_H_diag_list) - np.array(global_fci_list)), label='mu emb', linestyle='--', marker='x', color='r')

plt.xlabel('OH bond length (angstrom)')
plt.ylabel('Error (Ha)')
plt.grid(True)
plt.legend()
plt.savefig("test_error.png", dpi=300)
plt.show()

In [ ]:
# order by R_length

list_files_nonstretch = [ filename for filename in os.listdir(structure_dir) if filename.startswith('act_nonstretch_H2O')]

all_xyz_files_nonstretch = sorted(list_files_nonstretch, 
                       key=lambda text: int(text.partition("H2O_")[2].partition(".xyz")[0]))


list_file_paths_nonstretch = []
for xyz_filename in all_xyz_files_nonstretch:
    list_file_paths_nonstretch.append(os.path.join(structure_dir, xyz_filename))
list_file_paths_nonstretch

In [ ]:
global_dft_list2=[]
global_fci_list2=[]
global_H_diag_list2=[]
global_qubit_H_length_list2=[]
global_qubit_H_nqubits2=[]
global_H2 = []

huz_rhf_list2=[]
huz_ccsd_list2=[]
huz_fci_list2=[]
huz_H_diag_list2=[]
huz_qubit_H_length_list2=[]
huz_H_nqubits2=[]
huz_embed_H2 = []

mu_rhf_list2=[]
mu_ccsd_list2=[]
mu_fci_list2=[]
mu_H_diag_list2=[]
mu_qubit_H_length_list2=[]
mu_H_nqubits2=[]
mu_embed_H2 = []

N_active_MOs2 = []


for geometry_path in tqdm(list_file_paths_nonstretch):

    driver = NbedDriver(
            geometry=geometry_path,
            n_active_atoms=n_active_atoms,
            basis=basis,
            xc_functional=xc_functional,
            projector=projector,
            localization=localization,
            convergence=convergence,
            charge=charge,
            mu_level_shift=mu_level_shift,
            run_ccsd_emb=run_ccsd_emb,
            run_fci_emb=run_fci_emb,
            max_ram_memory=max_ram_memory,
            pyscf_print_level=pyscf_print_level,
            unit=unit,
            occupied_threshold=occupied_threshold,
            virtual_threshold=virtual_threshold,
            max_hf_cycles=max_hf_cycles,
            max_dft_cycles=max_dft_cycles,
            _init_huzinaga_rhf_with_mu = init_huzinaga_rhf_with_mu)


    ## global
    global_dft_list2.append(driver._global_rks.e_tot)
    global_fci_list2.append(driver._global_fci.e_tot)
    
    global_qubit_Ham = get_hamiltonian_from_driver(driver._global_hf,
                                           driver._global_hf.energy_nuc(), 
                                           transform)
    global_H2.append(global_qubit_Ham)
    global_H_sparse = get_sparse_operator(global_qubit_Ham)
    E_fci =  sp.sparse.linalg.eigsh(global_H_sparse, k=1, which='SA')[0][0] 
    global_H_diag_list2.append(E_fci)

    global_qubit_H_length_list2.append(len(global_qubit_Ham.terms))
    global_qubit_H_nqubits2.append(count_qubits(global_qubit_Ham))

    ##
    N_active_MOs2.append(len(driver.localized_system.active_MO_inds))
    
    # HUZ
    huz_rhf_list2.append(driver._huzinaga['e_rhf'])
    huz_ccsd_list2.append(driver._huzinaga['e_ccsd'])
    huz_fci_list2.append(driver._huzinaga['e_fci'])

    huz_classical = driver._huzinaga['classical_energy']
    huz_qubit_Ham = get_hamiltonian_from_driver(driver._huzinaga["scf"],
                                        huz_classical, #+ driver._huzinaga["scf"].energy_nuc(), 
                                        transform)
    
    huz_H_sparse = get_sparse_operator(huz_qubit_Ham)
    huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]
    huz_H_diag_list2.append(huz_e_emb)

    huz_qubit_H_length_list2.append(len(huz_qubit_Ham.terms))
    huz_nqubits = count_qubits(huz_qubit_Ham)
    huz_H_nqubits2.append(huz_nqubits)
    huz_embed_H2.append(huz_qubit_Ham)

    # mu shift
    mu_rhf_list2.append(driver._mu['e_rhf'])
    mu_ccsd_list2.append(driver._mu['e_ccsd'])
    mu_fci_list2.append(driver._mu['e_fci'])
    
    mu_classical = driver._mu['classical_energy']
    mu_qubit_Ham = get_hamiltonian_from_driver(driver._mu["scf"],
                                        mu_classical, #+ driver._mu["scf"].energy_nuc(), 
                                        transform)
    
    mu_H_sparse = get_sparse_operator(mu_qubit_Ham)
    mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]
    mu_H_diag_list2.append(mu_e_emb)
    
    mu_nqubits = count_qubits(mu_qubit_Ham)
    mu_qubit_H_length_list2.append(len(mu_qubit_Ham.terms))
    mu_H_nqubits2.append(mu_nqubits)
    mu_embed_H2.append(mu_qubit_Ham)
    
    del driver



In [ ]:
exp_data2 = {
    'global DFT': global_dft_list2, 
    
    'global FCI': global_fci_list2,
    'huz emb FCI': huz_H_diag_list2,
    'mu emb FCI': mu_H_diag_list2,
    
    'global H': global_qubit_H_length_list2,
    'huz H emb': huz_qubit_H_length_list2,
    'mu H emb': mu_qubit_H_length_list2,
    
    'global n qubits': global_qubit_H_nqubits2,
    'huz n qubits': huz_H_nqubits2,
    'mu n qubits': mu_H_nqubits2,
    'N active MOs': N_active_MOs2,
    
}

df2 = pd.DataFrame(data=exp_data2)

df2

In [ ]:
csv_filename2 = 'nonactive_OH_stretch_results_IBO.csv' 
df2.to_csv(os.path.join(current_dir, csv_filename2), index=False)

In [ ]:
exp_data2['bond_length'] =  [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]
exp_data2['global_H'] = [HamiltonianConverter(H)._intermediate for H in global_H2]
exp_data2['huz_embed_H'] = [HamiltonianConverter(H)._intermediate for H in huz_embed_H2]
exp_data2['mu_embed_H'] = [HamiltonianConverter(H)._intermediate for H in mu_embed_H2]


In [ ]:
df2.keys()

In [ ]:
# import results
csv_filename2 = 'nonactive_OH_stretch_results_IBO.csv'
df2 = pd.read_csv(os.path.join(current_dir, csv_filename2))

global_dft_list2 = np.array(df2['global DFT'])

global_fci_list2 = np.array(df2['global FCI'])
huz_H_diag_list2 = np.array(df2['huz emb FCI'])
mu_H_diag_list2 = np.array(df2['mu emb FCI'])

global_qubit_H_length_list2 = np.array(df2['global H'])
huz_qubit_H_length_list2 = np.array(df2['huz H emb'])
mu_qubit_H_length_list2 = np.array(df2['mu H emb'])

global_qubit_H_nqubits2 = np.array(df2['global n qubits'])
huz_H_nqubits2 = np.array(df2['huz n qubits'])
mu_H_nqubits2 = np.array(df2['mu n qubits'])
N_active_MOs2 = np.array(df2['N active MOs'])


In [ ]:
%matplotlib inline

R_list = [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]


plt.plot(R_list, global_dft_list2, label='global DFT', color='b')
plt.plot(R_list, global_fci_list2, label='global FCI', color='orange')
plt.plot(R_list, global_fci_list2, label='diag global H', linestyle='dashed', color='darkred')
plt.plot(R_list, huz_H_diag_list2, label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(R_list, mu_H_diag_list2, label='mu emb', linestyle='-', marker='x', color='r')

plt.xlabel('OH bond length (angstrom)')
plt.ylabel('energy (Ha)')
plt.grid(True)
plt.legend()
plt.savefig("test_energy2.png", dpi=300)
plt.show()

In [ ]:
%matplotlib inline

R_list = [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]


plt.plot(R_list, abs(np.array(global_dft_list2) - np.array(global_fci_list2)), label='global DFT', color='b', marker='+')
plt.plot(R_list, abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2)), label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(R_list, abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2)), label='mu emb', linestyle='--', marker='x', color='r')

plt.xlabel('OH bond length (angstrom)')
plt.ylabel('Error (Ha)')
plt.grid(True)
plt.legend()
plt.savefig("test_error2.png", dpi=300)
plt.show()

In [ ]:
%matplotlib inline

R_list = [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]


plt.plot(R_list, abs(np.array(global_dft_list2) - np.array(global_fci_list)), label='global DFT', color='b')
plt.plot(R_list, abs(np.array(huz_H_diag_list) - np.array(global_fci_list)), label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(R_list, abs(np.array(mu_H_diag_list) - np.array(global_fci_list)), label='mu emb', linestyle='-', marker='x', color='r')

plt.plot(R_list, abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2)), label='huz emb2', linestyle='dashed', marker='.', color='k')
plt.plot(R_list, abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2)), label='mu emb2', linestyle='-', marker='+', color='grey')


plt.xlabel('OH bond length (angstrom)')
plt.ylabel('Error (Ha)')
plt.grid(True)
plt.legend()
# plt.savefig("test_error.png", dpi=300)
plt.show()

In [ ]:
%matplotlib inline

R_list = [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]


plt.plot(R_list, abs(np.array(global_dft_list2) - np.array(global_fci_list)), label='global DFT', color='b')
plt.plot(R_list, abs(np.array(huz_H_diag_list) - np.array(global_fci_list)), label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(R_list, abs(np.array(mu_H_diag_list) - np.array(global_fci_list)), label='mu emb', linestyle='-', marker='x', color='r')

plt.plot(R_list, abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2)), label='huz emb2', linestyle='dashed', marker='.', color='k')
plt.plot(R_list, abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2)), label='mu emb2', linestyle='-', marker='+', color='grey')


plt.xlabel('OH bond length (angstrom)')
plt.ylabel('Error (Ha)')
plt.grid(True)
plt.legend()
# plt.savefig("test_error.png", dpi=300)
plt.show()

In [ ]:
with open(list_file_paths_nonstretch[0], 'r') as infile1:
    first_xyz = infile1.read()
    
with open(list_file_paths_nonstretch[-2], 'r') as infile2:
    last_xyz = infile2.read()
    
print(first_xyz)
print()
print()
print(last_xyz)

In [ ]:
with open(list_file_paths[0], 'r') as infile1:
    first_xyz = infile1.read()
    
with open(list_file_paths[-2], 'r') as infile2:
    last_xyz = infile2.read()
    
print(first_xyz)
print()
print()
print(last_xyz)

In [ ]:
figure, axis = plt.subplots(2, 2, dpi=300, sharex=True)

# 
axis[0, 0].xaxis.set_ticks(range(0,6,1))

# For first result
axis[0, 0].plot(R_list,abs(np.array(global_dft_list) - np.array(global_fci_list)), label='global DFT', color='g', marker='+')
axis[0, 0].plot(R_list,abs(np.array(huz_H_diag_list) - np.array(global_fci_list)), label='huz emb', linestyle='dashed', marker='o', color='b')
axis[0, 0].plot(R_list,abs(np.array(mu_H_diag_list) - np.array(global_fci_list)), label='mu emb', linestyle='-', marker='x', color='r')
axis[0, 0].set_ylabel('energy error (Ha)')
axis[0, 0].set_title("Result 1 - O-H stretch active")
  
# First result - number of terms
axis[1, 0].bar(np.array(R_list)-0.06, huz_qubit_H_length_list, width=0.06, color='b', align='center')
axis[1, 0].bar(R_list, mu_qubit_H_length_list, width=0.06, color='r', align='center')
axis[1, 0].bar(np.array(R_list)+0.06, global_qubit_H_length_list, width=0.06, color='g', align='center')
axis[1, 0].set_ylabel(r'$|H_{q}|$')
axis[1, 0].set_xlabel(r'Bond length ($\AA$)')
# axis[1, 0].set_title("Cosine Function")
  
# # 2nd result
axis[0, 1].plot(R_list,abs(np.array(global_dft_list2) - np.array(global_fci_list2)), label='global DFT', color='g', marker='+')
axis[0, 1].plot(R_list,abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2)), label='Huz emb', linestyle='dashed', marker='o', color='b')
axis[0, 1].plot(R_list,abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2)), label='Mu emb', linestyle='-', marker='x', color='r')
axis[0, 1].set_title("Result 2 - O-H fixed active")
  
# 2nd result - number of terms
axis[1, 1].bar(np.array(R_list)-0.06, huz_qubit_H_length_list2, width=0.06, color='b', align='center')
axis[1, 1].bar(R_list, mu_qubit_H_length_list2, width=0.075, color='r', align='center')
axis[1, 1].bar(np.array(R_list)+0.06, global_qubit_H_length_list2, width=0.06, color='g', align='center')
axis[1, 1].set_xlabel(r'Bond length ($\AA$)')

# shared y axis
axis[0, 0].get_shared_y_axes().join(axis[0, 0], axis[0, 1])
axis[1, 0].get_shared_y_axes().join(axis[1, 0], axis[1, 1])

# remove rightmost y labels
axis[0, 1].set_yticklabels([])
axis[1, 1].set_yticklabels([])

axis[0, 1].legend(loc="upper right")

figure.savefig('water_correlated_results_old2.png', dpi=300)

# figure.show()

In [ ]:
np.array(global_dft_list2) - np.array(global_dft_list)

In [ ]:
# geometry_path  = list_file_paths[3]
# projector = 'both'#'both', 'huzinaga'  

# driver = NbedDriver(
#         geometry=geometry_path,
#         n_active_atoms=n_active_atoms,
#         basis=basis,
#         xc_functional=xc_functional,
#         projector=projector,
#         localization=localization,
#         convergence=convergence,
#         charge=charge,
#         mu_level_shift=mu_level_shift,
#         run_ccsd_emb=run_ccsd_emb,
#         run_fci_emb=run_fci_emb,
#         max_ram_memory=max_ram_memory,
#         pyscf_print_level=pyscf_print_level,
#         unit=unit,
#         occupied_threshold=occupied_threshold,
#         virtual_threshold=virtual_threshold,
#         max_hf_cycles=max_hf_cycles)

In [ ]:
# mu_scf = driver._mu["scf"]
# huz_scf = driver._huzinaga["scf"]
# global_scf = driver._global_hf

In [ ]:
# c_mat = mu_scf.mo_coeff
# mo_occ = mu_scf.get_occ()
# dmat = mu_scf.make_rdm1(mo_coeff=c_mat, mo_occ=mo_occ)
# F =  mu_scf.get_fock(dm=dmat)

# print(np.around(c_mat.T @ F @c_mat,
#          3))

In [ ]:
# c_mat = huz_scf.mo_coeff
# mo_occ = huz_scf.get_occ()
# dmat = huz_scf.make_rdm1(mo_coeff=c_mat, mo_occ=mo_occ)
# F =  huz_scf.get_fock(dm=dmat)

# print(np.around(c_mat.T @ F @c_mat,
#          3))

In [ ]:
fig = plt.figure(dpi=300)

gs = fig.add_gridspec(3,2)

ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[2,:]) # goes over both columns!

# For first result
ax1.plot(R_list,abs(np.array(global_dft_list) - np.array(global_fci_list)), label='global DFT', color='g', marker='+')
ax1.plot(R_list,abs(np.array(huz_H_diag_list) - np.array(global_fci_list)), label='huz emb', linestyle='dashed', marker='o', color='b')
ax1.plot(R_list,abs(np.array(mu_H_diag_list) - np.array(global_fci_list)), label='mu emb', linestyle='-', marker='x', color='r')
ax1.set_ylabel('energy error (Ha)')
ax1.set_title("Result 1 - O-H stretch active")

# First result - number of terms
ax3.bar(np.array(R_list)-0.06, huz_qubit_H_length_list, width=0.06, color='b', align='center')
ax3.bar(R_list, mu_qubit_H_length_list, width=0.06, color='r', align='center')
ax3.bar(np.array(R_list)+0.06, global_qubit_H_length_list, width=0.06, color='g', align='center')
ax3.set_ylabel(r'$|H_{q}|$')
ax3.set_xlabel(r'Bond length ($\AA$)')
# axis[1, 0].set_title("Cosine Function")

# # 2nd result
ax2.plot(R_list,abs(np.array(global_dft_list2) - np.array(global_fci_list2)), label='global DFT', color='g', marker='+')
ax2.plot(R_list,abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2)), label='Huz emb', linestyle='dashed', marker='o', color='b')
ax2.plot(R_list,abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2)), label='Mu emb', linestyle='-', marker='x', color='r')
ax2.set_title("Result 2 - O-H fixed active")
  
# 2nd result - number of terms
ax4.bar(np.array(R_list)-0.06, huz_qubit_H_length_list2, width=0.06, color='b', align='center')
ax4.bar(R_list, mu_qubit_H_length_list2, width=0.075, color='r', align='center')
ax4.bar(np.array(R_list)+0.06, global_qubit_H_length_list2, width=0.06, color='g', align='center')
ax4.set_xlabel(r'Bond length ($\AA$)')

# shared y axis
ax1.get_shared_y_axes().join(ax1, ax2)
ax3.get_shared_y_axes().join(ax3, ax4)

# shared x axis
ax1.get_shared_x_axes().join(ax1, ax3)
ax2.get_shared_x_axes().join(ax2, ax4)


# remove rightmost y labels
ax2.set_yticklabels([])
ax4.set_yticklabels([])

# remove rightmost x labels
ax1.set_xticklabels([])
ax2.set_xticklabels([])

ax5.plot(R_list,abs(np.array(huz_H_diag_list) - np.array(global_fci_list)), label='huz emb', linestyle='dashed', marker='o', color='b')
ax5.plot(R_list,abs(np.array(mu_H_diag_list) - np.array(global_fci_list)), label='mu emb', linestyle='-', marker='x', color='r')
ax5.plot(R_list,abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2)), label='Huz emb', linestyle='dashed', marker='o', color='b')
ax5.plot(R_list,abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2)), label='Mu emb', linestyle='-', marker='x', color='r')

# ax5.plot(R_list,abs(np.array(global_dft_list2) - np.array(global_fci_list2)), label='global DFT', color='g', marker='+')
# ax5.plot(R_list,abs(np.array(global_dft_list) - np.array(global_fci_list)), label='global DFT', color='r', marker='x', linestyle='--')

ax5.set_ylabel('energy error (Ha)')

In [ ]:
fig = plt.figure(dpi=300)

gs = fig.add_gridspec(3,2)

ax1 = fig.add_subplot(gs[0,:])# goes over both columns!
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])


ax1.plot(R_list,abs(np.array(huz_H_diag_list) - np.array(global_fci_list)), label='Huz emb - act stretch', marker='o',
         color='crimson', linestyle='dashed')
ax1.plot(R_list,abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2)), label='Huz emb - env stretch', marker='s', 
         color='navy', linestyle='dashed')
ax1.plot(R_list,abs(np.array(mu_H_diag_list) - np.array(global_fci_list)), label='mu emb - act stretch', marker='x', 
         color='lightcoral',linestyle='')
ax1.plot(R_list,abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2)), label='Mu emb - env stretch', marker='+', 
         color='cornflowerblue', linestyle='')

ax1.plot(R_list,abs(np.array(global_dft_list2) - np.array(global_fci_list2)), label='global DFT - env stretch', 
         color='darkgreen', marker='1')
ax1.plot(R_list,abs(np.array(global_dft_list) - np.array(global_fci_list)), label='global DFT - act stretch', 
         color='lime', marker='2', linestyle='dotted')


# ax1.legend(loc="upper right", fontsize =5)

ax1.set_ylabel('energy error (Ha)')
ax1.set_xlabel(r'Bond length ($\AA$)')


# First result - number of terms
ax2.bar(np.array(R_list)-0.06, huz_qubit_H_length_list, width=0.06, color='crimson', align='center', label='$|H_{huz}|$-act stretch')
ax2.bar(R_list, mu_qubit_H_length_list, width=0.06, color='lightcoral', align='center', label='$|H_{mu}|$ - act stretch')
ax2.bar(np.array(R_list)+0.06, global_qubit_H_length_list, width=0.06, color='k', align='center', label='$|H_{FCI}|$')
ax2.set_ylabel(r'$|H_{q}|$')
ax2.set_xlabel(r'Bond length ($\AA$)')
# axis[1, 0].set_title("Cosine Function")
  
# 2nd result - number of terms
ax3.bar(np.array(R_list)-0.06, huz_qubit_H_length_list2, width=0.06, color='navy', align='center', label='$|H_{huz}|$- env stretch')
ax3.bar(R_list, mu_qubit_H_length_list2, width=0.075, color='cornflowerblue', align='center', label='$|H_{mu}|$ - env stretch')
ax3.bar(np.array(R_list)+0.06, global_qubit_H_length_list2, width=0.06, color='k', align='center', label='$|H_{FCI}|$')
ax3.set_xlabel(r'Bond length ($\AA$)')

# shared y axis
ax2.get_shared_y_axes().join(ax2, ax3)

# remove rightmost y labels
ax3.set_yticklabels([])

plt.tight_layout()
ax1.legend(bbox_to_anchor=(0.5, 0.5, 0.5, 0.5), fontsize=6)
ax2.legend(bbox_to_anchor=(0.0, 0.75, 0.5, 0.5), fontsize=6)
ax3.legend(bbox_to_anchor=(0.0, 0.75, 0.5, 0.5), fontsize=6)

fig.savefig('water_correlated_results_new.png', bbox_inches = 'tight')
plt.show()


https://www.ucl.ac.uk/cam/brand/guidelines/colour

In [ ]:
fig = plt.figure(dpi=300)

gs = fig.add_gridspec(3,2)

ax1 = fig.add_subplot(gs[0,:])# goes over both columns!
ax2 = fig.add_subplot(gs[1,:])
# ax3 = fig.add_subplot(gs[1,1])

# colour_stone = '#C0BDB0' #'#D6D2C4'
# colour_dark_brown = '#4E3629'

# colour_stone = '#500778' 
# colour_dark_brown = '#AC145A'

colour_stone = '#8C8279' 
colour_dark_brown = 'k'#'#47423e'

colour_midblue = '#002855'
colour_brightblue = '#0097A9'

colour_orange = '#EA7600'
colour_dark_yellow = '#F6BE00'

FCI_colour = '#002855'


ax1.plot(R_list,np.log10(abs(np.array(huz_H_diag_list) - np.array(global_fci_list))), label='Huz emb - act stretch', marker='o',
         color=colour_stone, linestyle='dashed')
ax1.plot(R_list,np.log10(abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2))), label='Huz emb - env stretch', marker='s', 
         color=colour_dark_brown, linestyle='dashed')
ax1.plot(R_list,np.log10(abs(np.array(mu_H_diag_list) - np.array(global_fci_list))), label='mu emb - act stretch', marker='x', 
         color=colour_orange,linestyle='')
ax1.plot(R_list, np.log10(abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2))), label='Mu emb - env stretch', marker='+', 
         color=colour_dark_yellow, linestyle='')

ax1.plot(R_list,np.log10(abs(np.array(global_dft_list2) - np.array(global_fci_list2))), label='global DFT - env stretch', 
         color=colour_midblue, marker='1')
ax1.plot(R_list,np.log10(abs(np.array(global_dft_list) - np.array(global_fci_list))), label='global DFT - act stretch', 
         color=colour_brightblue, marker='2', linestyle='dotted')


# ax1.legend(loc="upper right", fontsize =5)

# ax1.set_ylabel('energy error (Ha)')
ax1.set_ylabel(r'$log_{10}(\frac{|\Delta E|}{Ha})$')
# ax1.set_xlabel(r'Bond length ($\AA$)')
ax1.set_xticklabels([])


# First result - number of terms
wid = 0.035

ax2.bar(np.array(R_list)-2*wid, huz_qubit_H_length_list, width=wid, color=colour_stone, align='center', label='$|H_{huz}|$-act stretch', edgecolor='k', linewidth=0.5)
ax2.bar(np.array(R_list)-wid, mu_qubit_H_length_list, width=wid, color=colour_orange, align='center', label='$|H_{mu}|$ - act stretch', edgecolor='k', linewidth=0.5)
ax2.bar(np.array(R_list), global_qubit_H_length_list, width=wid, color=FCI_colour, align='center', label='$|H_{FCI}|$', edgecolor='k', linewidth=0.5)
ax2.set_ylabel(r'$|H_{q}|$')
ax2.set_xlabel(r'Bond length ($\AA$)')

ax2.bar(np.array(R_list)+2*wid, huz_qubit_H_length_list2, width=wid, color=colour_dark_brown, align='center', label='$|H_{huz}|$- env stretch', edgecolor='k', linewidth=0.5)
ax2.bar(np.array(R_list)+wid, mu_qubit_H_length_list2, width=wid, color=colour_dark_yellow, align='center', label='$|H_{mu}|$ - env stretch', edgecolor='k', linewidth=0.5)
ax2.set_xlabel(r'Bond length ($\AA$)')

# shared x axis
ax1.get_shared_x_axes().join(ax1, ax2)

# set axis limits
# ax1.yaxis.set_ticks(np.arange(0, 0.4, 0.1))
ax1.yaxis.set_ticks(np.arange(-2.5, 0, 1))
ax2.yaxis.set_ticks(np.arange(0, 5000, 1000))

plt.tight_layout()
# ax1.legend(bbox_to_anchor=(0.7, 0.5, 0.5, 0.5), fontsize=6)
ax1.legend(bbox_to_anchor=(0.5, -0.2, 0.5, 0.5), fontsize=6)
ax2.legend(bbox_to_anchor=(0.25, 0.65, 0.5, 0.5), fontsize=6)

fig.savefig('water_correlated_results_IBO.png', bbox_inches = 'tight')
plt.show()


In [ ]:


R_list =  [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]

error_DFT = abs(np.array(global_dft_list) - np.array(global_fci_list))
error_HUZ = abs(np.array(huz_H_diag_list) - np.array(global_fci_list))
error_MU = abs(np.array(mu_H_diag_list) - np.array(global_fci_list))



In [ ]:
plot_data_1 = {
    'OH length': R_list,
    'error DFT': error_DFT, 
    'error HUZ': error_HUZ,
    'error mu': error_MU,
    
    'Hfull': global_qubit_H_length_list,
    'Hhuz': huz_qubit_H_length_list,
    'Hmu': mu_qubit_H_length_list,
    
    'global n qubits': global_qubit_H_nqubits,
    'huz n qubits': huz_H_nqubits,
    'mu n qubits': mu_H_nqubits,
    'N active MOs': N_active_MOs,
}

df_data1= pd.DataFrame(data=plot_data_1)
df_data1

In [ ]:
print(df_data1.to_latex(index=False))

In [ ]:
error_DFT2 = abs(np.array(global_dft_list2) - np.array(global_fci_list2))
error_HUZ2 = abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2))
error_MU2 = abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2))


plot_data_2 = {
    'OH length': R_list,
    'error DFT': error_DFT2, 
    'error HUZ': error_HUZ2,
    'error mu': error_MU2,
    
    'Hfull': global_qubit_H_length_list2,
    'Hhuz': huz_qubit_H_length_list2,
    'Hmu': mu_qubit_H_length_list2,
    
    'global n qubits': global_qubit_H_nqubits2,
    'huz n qubits': huz_H_nqubits2,
    'mu n qubits': mu_H_nqubits2,
    'N active MOs': N_active_MOs2,
}

df_data2= pd.DataFrame(data=plot_data_2)
df_data2

In [ ]:
print(df_data2.to_latex(index=False))

In [ ]:
%matplotlib inline

R_list = [0.4, 0.6, 0.7989538439284714, 1, 1.2, 1.5, 2, 3, 4, 5]


plt.plot(R_list, abs(np.array(global_dft_list2) - np.array(global_fci_list)), label='global DFT', color='b')
plt.plot(R_list, abs(np.array(huz_H_diag_list) - np.array(global_fci_list)), label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(R_list, abs(np.array(mu_H_diag_list) - np.array(global_fci_list)), label='mu emb', linestyle='-', marker='x', color='r')

plt.plot(R_list, abs(np.array(huz_H_diag_list2) - np.array(global_fci_list2)), label='huz emb2', linestyle='dashed', marker='.', color='k')
plt.plot(R_list, abs(np.array(mu_H_diag_list2) - np.array(global_fci_list2)), label='mu emb2', linestyle='-', marker='+', color='grey')


plt.xlabel('OH bond length (angstrom)')
plt.ylabel('Error (Ha)')
plt.grid(True)
plt.legend()
# plt.savefig("test_error.png", dpi=300)
plt.show()

In [ ]:
##
pickle_dir = os.path.join(current_dir, "pickle_output")
# os.makedirs(pickle_dir)

In [ ]:
fname = 'active_stretch_IBO'
fpath = os.path.join(pickle_dir, fname+'.pickle')
with open(fpath, 'wb') as fp:
    pickle.dump(exp_data, fp)

In [ ]:
fname2 = 'enviro_stretch_IBO'
fpath2 = os.path.join(pickle_dir, fname2+'.pickle')
with open(fpath2, 'wb') as fp:
    pickle.dump(exp_data2, fp)